In [101]:
import numpy as np
import pandas as pd
from datetime import datetime

In [102]:
# Read dataset
df = pd.read_csv("Data-GP1-1.csv")
df

Mon  Tue  Wed  Thu    Date  Stormy  Mixed         p         q  Rainy  \
0      1    0    0    0  911202       1      0 -0.430783  8.994421      1   
1      0    1    0    0  911203       1      0  0.000000  7.707063      0   
2      0    0    1    0  911204       0      1  0.072321  8.350194      1   
3      0    0    0    1  911205       1      0  0.247139  8.656955      0   
4      0    0    0    0  911206       1      0  0.664327  7.844241      0   
..   ...  ...  ...  ...     ...     ...    ...       ...       ...    ...   
106    1    0    0    0  920504       0      0 -0.798508  8.610683      0   
107    0    1    0    0  920505       0      1 -0.087011  7.162397      0   
108    0    0    1    0  920506       0      1  0.184922  7.362010      0   
109    0    0    0    1  920507       0      1  0.223143  8.764053      0   
110    0    0    0    0  920508       0      1  0.561118  8.328451      0   

     Cold      Wind  
0       0  2.995732  
1       0  2.995732  
2       1  2.813411  
3       1  3.036554  
4       1  3.036554  
..    ...       ...  
106     0  2.862201  
107     0  2.908721  
108     0  2.862201  
109     0  2.813411  
110     0  2.862201  

[111 rows x 12 columns]

In [103]:
# Check for missing data (NA or NaN values)
missing_data = df.isna()  # or df.isnull()

# Check if there are any missing values in the entire DataFrame
any_missing = missing_data.any().any()

print("Missing Value Counts")
print(missing_counts)

Missing Value Counts
Mon       0
Tue       0
Wed       0
Thu       0
Date      0
Jan       0
Feb       0
Mar       0
Apr       0
May       0
Month     0
Year      0
Stormy    0
Mixed     0
p         0
q         0
Rainy     0
Cold      0
Wind      0
dtype: int64


In [104]:
# Original column headers
og_columns = df.columns

weekday_list = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
month_list = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
monthnum_list = [1,2,3,4,5,6,7,8,9,10,11,12]

# Set columns "Fri", "Sat" and "Sun" with 0 values
df["Fri"] = 0
df["Sat"] = 0
df["Sun"] = 0

# Apply datetime operations to convert to weekday
df["Day"] = pd.to_datetime(df["Date"], format="%y%m%d").apply(lambda x: weekday_list[x.weekday()])

# Extract "Date", "Month" and "Year" and save values as int type
df["Year"] = (df["Date"].astype(str).str[:2]).astype(int)
df["Month"] = (df["Date"].astype(str).str[2:4]).astype(int)
df["Date"] = (df["Date"].astype(str).str[4:]).astype(int)

# Set columns of "Jan" to "Dec" with 0 values
for month in month_list:
    df[month] = 0

# Rearranging columns within df for better visualization
new_columns = og_columns[:4].tolist()
new_columns.extend(["Fri","Sat","Sun","Day"]) 
new_columns.extend([og_columns[4]])
new_columns.extend(month_list)
new_columns.extend(["Month", "Year"])
new_columns.extend(og_columns[5:])

df = df.reindex(columns=new_columns)
df


Mon  Tue  Wed  Thu  Fri  Sat  Sun  Day  Date  Jan  ...  Dec  Month  Year  \
0      1    0    0    0    0    0    0  Mon     2    0  ...    0     12    91   
1      0    1    0    0    0    0    0  Tue     3    0  ...    0     12    91   
2      0    0    1    0    0    0    0  Wed     4    0  ...    0     12    91   
3      0    0    0    1    0    0    0  Thu     5    0  ...    0     12    91   
4      0    0    0    0    0    0    0  Fri     6    0  ...    0     12    91   
..   ...  ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...    ...   ...   
106    1    0    0    0    0    0    0  Mon     4    0  ...    0      5    92   
107    0    1    0    0    0    0    0  Tue     5    0  ...    0      5    92   
108    0    0    1    0    0    0    0  Wed     6    0  ...    0      5    92   
109    0    0    0    1    0    0    0  Thu     7    0  ...    0      5    92   
110    0    0    0    0    0    0    0  Fri     8    0  ...    0      5    92   

     Stormy  Mixed         p         q  Rainy  Cold      Wind  
0         1      0 -0.430783  8.994421      1     0  2.995732  
1         1      0  0.000000  7.707063      0     0  2.995732  
2         0      1  0.072321  8.350194      1     1  2.813411  
3         1      0  0.247139  8.656955      0     1  3.036554  
4         1      0  0.664327  7.844241      0     1  3.036554  
..      ...    ...       ...       ...    ...   ...       ...  
106       0      0 -0.798508  8.610683      0     0  2.862201  
107       0      1 -0.087011  7.162397      0     0  2.908721  
108       0      1  0.184922  7.362010      0     0  2.862201  
109       0      1  0.223143  8.764053      0     0  2.813411  
110       0      1  0.561118  8.328451      0     0  2.862201  

[111 rows x 30 columns]

In [105]:
# Update One-Hot Encoding for the months
for month, monthnum in zip(month_list, monthnum_list):
    df[month] = (df["Month"] == monthnum).astype(int)
    
# Update One-Hot Encoding for Fri, Sat and Sun
for day in ["Fri","Sat", "Sun"]:
    df[day] = (df["Day"] == day).astype(int)

# Check if the One-Hot Encoding is done correctly
df["Date Error"] = (
    (df["Day"].isin(weekday_list)) &  # Check if "Day" contains valid weekdays
    (df[weekday_list].sum(axis=1) != 1)  # Check if exactly one day is encoded as 1
)
    
incorrect_encoding = df[df["Date Error"] == True]

# If One Hot Date Encoding is done correctly, drop all irrelavant columns and download csv
if incorrect_encoding.empty:
    df.drop("Date Error", axis=1, inplace=True)
    df.drop("Day", axis=1, inplace=True)
    df.drop("Fri", axis=1, inplace=True)
    df.drop("Sat", axis=1, inplace=True)
    df.drop("Sun", axis=1, inplace=True)
    
    for month in month_list:
        if (df[month] == 0).all():
            df.drop(month, axis=1, inplace=True)
    
    df.drop("Dec", axis=1, inplace=True)

    # Specify the file path to save the CSV file
    file_path = "./Data-GP1-1(updated).csv"

    # Save the DataFrame as a CSV file
    df.to_csv(file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV file

    print(f"DataFrame saved as {file_path}")
else:
    # Print out rows with date errors
    print(incorrect_encoding)

DataFrame saved as ./Data-GP1-1(updated).csv


In [107]:
# Check if csv file is downloaded properly.
df = pd.read_csv("Data-GP1-1(updated).csv")

# Calculate the sum of each column and assign it to a new row
df.loc['Total'] = df.sum()
df

Mon   Tue   Wed   Thu    Date   Jan   Feb   Mar   Apr  May  Month  \
0       1.0   0.0   0.0   0.0     2.0   0.0   0.0   0.0   0.0  0.0   12.0   
1       0.0   1.0   0.0   0.0     3.0   0.0   0.0   0.0   0.0  0.0   12.0   
2       0.0   0.0   1.0   0.0     4.0   0.0   0.0   0.0   0.0  0.0   12.0   
3       0.0   0.0   0.0   1.0     5.0   0.0   0.0   0.0   0.0  0.0   12.0   
4       0.0   0.0   0.0   0.0     6.0   0.0   0.0   0.0   0.0  0.0   12.0   
...     ...   ...   ...   ...     ...   ...   ...   ...   ...  ...    ...   
107     0.0   1.0   0.0   0.0     5.0   0.0   0.0   0.0   0.0  1.0    5.0   
108     0.0   0.0   1.0   0.0     6.0   0.0   0.0   0.0   0.0  1.0    5.0   
109     0.0   0.0   0.0   1.0     7.0   0.0   0.0   0.0   0.0  1.0    5.0   
110     0.0   0.0   0.0   0.0     8.0   0.0   0.0   0.0   0.0  1.0    5.0   
Total  21.0  23.0  21.0  23.0  1701.0  22.0  19.0  22.0  21.0  6.0  492.0   

          Year  Stormy  Mixed          p           q  Rainy  Cold        Wind  
0         91.0     1.0    0.0  -0.430783    8.994421    1.0   0.0    2.995732  
1         91.0     1.0    0.0   0.000000    7.707063    0.0   0.0    2.995732  
2         91.0     0.0    1.0   0.072321    8.350194    1.0   1.0    2.813411  
3         91.0     1.0    0.0   0.247139    8.656955    0.0   1.0    3.036554  
4         91.0     1.0    0.0   0.664327    7.844241    0.0   1.0    3.036554  
...        ...     ...    ...        ...         ...    ...   ...         ...  
107       92.0     0.0    1.0  -0.087011    7.162397    0.0   0.0    2.908721  
108       92.0     0.0    1.0   0.184922    7.362010    0.0   0.0    2.862201  
109       92.0     0.0    1.0   0.223143    8.764053    0.0   0.0    2.813411  
110       92.0     0.0    1.0   0.561118    8.328451    0.0   0.0    2.862201  
Total  10191.0    32.0   34.0 -21.498606  946.100691   18.0  56.0  317.759182  

[112 rows x 19 columns]